In [2]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 7.1 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
data = yf.download("GC=F", start='2017-07-22', end='2022-07-22')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-07-24,1256.000000,1256.000000,1253.900024,1253.900024,1253.900024,8
2017-07-25,1252.699951,1253.599976,1249.199951,1251.699951,1251.699951,18
2017-07-26,1244.800049,1249.000000,1244.800049,1249.000000,1249.000000,7
2017-07-27,1259.599976,1259.599976,1259.599976,1259.599976,1259.599976,220309
2017-07-28,1258.400024,1270.000000,1256.900024,1268.400024,1268.400024,64180
...,...,...,...,...,...,...
2022-07-15,1705.400024,1709.699951,1701.099976,1702.400024,1702.400024,721
2022-07-18,1712.199951,1712.400024,1709.199951,1709.199951,1709.199951,264
2022-07-19,1712.300049,1714.400024,1706.099976,1710.000000,1710.000000,1684


In [5]:
df = data['Adj Close'].pct_change() * 100
df

Date
2017-07-24         NaN
2017-07-25   -0.175458
2017-07-26   -0.215703
2017-07-27    0.848677
2017-07-28    0.698638
                ...   
2022-07-15   -0.123202
2022-07-18    0.399432
2022-07-19    0.046808
2022-07-20   -0.614035
2022-07-21    0.776696
Name: Adj Close, Length: 1258, dtype: float64

In [6]:
df = df.rename("Today")
df

Date
2017-07-24         NaN
2017-07-25   -0.175458
2017-07-26   -0.215703
2017-07-27    0.848677
2017-07-28    0.698638
                ...   
2022-07-15   -0.123202
2022-07-18    0.399432
2022-07-19    0.046808
2022-07-20   -0.614035
2022-07-21    0.776696
Name: Today, Length: 1258, dtype: float64

In [7]:
df = df.reset_index()
df

,Date,Today
0,2017-07-24,NaN
1,2017-07-25,-0.175458
2,2017-07-26,-0.215703
3,2017-07-27,0.848677
4,2017-07-28,0.698638
...,...,...
1253,2022-07-15,-0.123202
1254,2022-07-18,0.399432
1255,2022-07-19,0.046808
1256,2022-07-20,-0.614035


In [8]:
for i in range(1,6):
    df['Lag '+str(i)] = df['Today'].shift(i)
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,2017-07-24,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-25,-0.175458,NaN,NaN,NaN,NaN,NaN
2,2017-07-26,-0.215703,-0.175458,NaN,NaN,NaN,NaN
3,2017-07-27,0.848677,-0.215703,-0.175458,NaN,NaN,NaN
4,2017-07-28,0.698638,0.848677,-0.215703,-0.175458,NaN,NaN
...,...,...,...,...,...,...,...
1253,2022-07-15,-0.123202,-1.712602,0.632502,-0.387280,-0.608984,0.155357
1254,2022-07-18,0.399432,-0.123202,-1.712602,0.632502,-0.387280,-0.608984
1255,2022-07-19,0.046808,0.399432,-0.123202,-1.712602,0.632502,-0.387280
1256,2022-07-20,-0.614035,0.046808,0.399432,-0.123202,-1.712602,0.632502


In [9]:
df['Volume'] = data.Volume.shift(1).values/1000
df = df.dropna()
df

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
6,2017-08-01,0.473709,-0.141915,0.698638,0.848677,-0.215703,-0.175458,7.780
7,2017-08-02,-0.062858,0.473709,-0.141915,0.698638,0.848677,-0.215703,1.061
8,2017-08-03,-0.314515,-0.062858,0.473709,-0.141915,0.698638,0.848677,0.393
9,2017-08-04,-0.749330,-0.314515,-0.062858,0.473709,-0.141915,0.698638,0.878
10,2017-08-07,-0.007955,-0.749330,-0.314515,-0.062858,0.473709,-0.141915,0.521
...,...,...,...,...,...,...,...,...
1253,2022-07-15,-0.123202,-1.712602,0.632502,-0.387280,-0.608984,0.155357,1.289
1254,2022-07-18,0.399432,-0.123202,-1.712602,0.632502,-0.387280,-0.608984,0.721
1255,2022-07-19,0.046808,0.399432,-0.123202,-1.712602,0.632502,-0.387280,0.264
1256,2022-07-20,-0.614035,0.046808,0.399432,-0.123202,-1.712602,0.632502,1.684


In [10]:
df['Direction'] = [1 if i>0 else 0 for i in df['Today']]
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction
6,2017-08-01,0.473709,-0.141915,0.698638,0.848677,-0.215703,-0.175458,7.780,1
7,2017-08-02,-0.062858,0.473709,-0.141915,0.698638,0.848677,-0.215703,1.061,0
8,2017-08-03,-0.314515,-0.062858,0.473709,-0.141915,0.698638,0.848677,0.393,0
9,2017-08-04,-0.749330,-0.314515,-0.062858,0.473709,-0.141915,0.698638,0.878,0
10,2017-08-07,-0.007955,-0.749330,-0.314515,-0.062858,0.473709,-0.141915,0.521,0
...,...,...,...,...,...,...,...,...,...
1253,2022-07-15,-0.123202,-1.712602,0.632502,-0.387280,-0.608984,0.155357,1.289,0
1254,2022-07-18,0.399432,-0.123202,-1.712602,0.632502,-0.387280,-0.608984,0.721,1
1255,2022-07-19,0.046808,0.399432,-0.123202,-1.712602,0.632502,-0.387280,0.264,1
1256,2022-07-20,-0.614035,0.046808,0.399432,-0.123202,-1.712602,0.632502,1.684,0


In [11]:
x_train = df[df.Date.dt.year< 2022][['Lag 1','Lag 2']]
y_train = df[df.Date.dt.year< 2022]['Direction']
x_test = df[df.Date.dt.year == 2022][['Lag 1','Lag 2']]
y_test = df[df.Date.dt.year == 2022]['Direction']

In [12]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(x_train,y_train)

LogisticRegression()

In [13]:
predictions = logmodel.predict(x_test)

In [14]:
y_test

1120    0
1121    1
1122    1
1123    0
1124    1
       ..
1253    0
1254    1
1255    1
1256    0
1257    1
Name: Direction, Length: 138, dtype: int64

In [15]:
x_test

,Lag 1,Lag 2
1120,0.816464,0.421028
1121,-1.537618,0.816464
1122,0.811380,-1.537618
1123,0.584343,0.811380
1124,-1.967556,0.584343
...,...,...
1253,-1.712602,0.632502
1254,-0.123202,-1.712602
1255,0.399432,-0.123202
1256,0.046808,0.399432


In [16]:
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [17]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[ 3, 62],
       [ 1, 72]])

In [18]:
from sklearn.metrics import classification_report

In [19]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.75      0.05      0.09        65
           1       0.54      0.99      0.70        73

    accuracy                           0.54       138
   macro avg       0.64      0.52      0.39       138
weighted avg       0.64      0.54      0.41       138

